In [1]:
# Import libraries
## for profiling
from profiler.main import profile

# for titiler-xarray
from typing import Any
from zarr_reader import xarray_open_dataset, get_variable
import morecantile
from xarray_tile_reader import XarrayTileReader

# for titiler-pgstac
from titiler.pgstac.mosaic import PGSTACBackend
from typing import Any, Dict, List, Tuple
from rio_tiler.mosaic import mosaic_reader
from rio_tiler.models import ImageData
from geojson_pydantic import Polygon

# Only if wanting to see a picture
import io
from PIL import Image

import os
os.environ['AWS_PROFILE'] = 'veda-dev'

In [2]:
xyz_tile = (0,0,0)

## Profile titiler-pgstac

In [3]:
import pgstac.profile_pgstac as profile_pgstac 

In [4]:
image_and_assets, logs = profile_pgstac.tile(*xyz_tile)
logs

id='e0ccb2a67a881fa72642675a9a91f29c' input_search={'bbox': [-180.0, -90.0, 180.0, 90.0], 'limit': 100, 'filter': {'op': 'or', 'args': [{'op': 't_intersects', 'args': [{'property': 'datetime'}, {'interval': ['1950-04-01T00:00:00Z']}]}, {'op': 't_intersects', 'args': [{'property': 'datetime'}, {'interval': ['1951-04-01T00:00:00Z']}]}], 'limit': 100}, 'collections': ['CMIP6_ensemble_median_TAS']} sql_where="collection = ANY ('{CMIP6_ensemble_median_TAS}')  AND st_intersects(geometry, '0103000020E6100000010000000500000000000000008066C000000000008056C000000000008066C0000000000080564000000000008066400000000000805640000000000080664000000000008056C000000000008066C000000000008056C0') AND ((datetime <= '1950-04-01 00:00:00+00'::timestamptz AND end_datetime >= '1950-04-01 00:00:00+00'::timestamptz) OR (datetime <= '1951-04-01 00:00:00+00'::timestamptz AND end_datetime >= '1951-04-01 00:00:00+00'::timestamptz))" orderby='datetime DESC, id DESC' lastused=datetime.datetime(2023, 6, 2, 17, 46, 33, 1

{'cprofile': ['   ncalls  tottime  percall  cumtime  percall filename:lineno(function)',
  "       11    3.787    0.344    3.787    0.344 {method 'acquire' of '_thread.lock' objects}",
  '        4    0.094    0.023    0.095    0.024 connection.py:949(wait)',
  '        2    0.001    0.000    0.001    0.000 core.py:3433(__setmask__)',
  '        1    0.001    0.001    3.887    3.887 profile_pgstac.py:136(tile)',
  '        8    0.001    0.000    0.001    0.000 connection.py:434(_exec_command)']}

## Profile titiler-xarray

In [ ]:
@profile(add_to_return=True, quiet=False, log_library='s3fs')
def tile(src_path: str, x: int, y: int, z: int, *, variable: str, time_slice: str = None, **kwargs: Any):

    with xarray_open_dataset(
        src_path,
        group=z,
        decode_times=False,
        **kwargs,
    ) as dataset:
        dataarray = get_variable(dataset, variable=variable, time_slice=time_slice)
        
        with XarrayTileReader(dataarray) as src_dst:
            return src_dst.tile(x, y, z)

In [ ]:
image_and_assets, logs = tile(
    "s3://power-analysis-ready-datastore/power_901_monthly_meteorology_utc.zarr", 
    *xyz_tile,
    reference=False,
    variable="TS",
)
# logs

In [ ]:
content = image_and_assets.render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im

In [ ]:
content = image_and_assets[0].render(
    img_format='PNG'
)

im = Image.open(io.BytesIO(content))
im